In [37]:
import csv
import re
import math

import plotly.graph_objects as go

from collections import Counter

In [38]:
def find_query(events):
    """
    Поиск слов не являющихся префиксом для любого другого слова в списке
    """
    queries = []
    for e in events:
        q = events[:]
        q.remove(e)
        if e not in ''.join(q):
            queries.append(e.capitalize())
    return queries

In [39]:
categories = ['Продукты', 'Где поесть', 'Гостиница', 'Достопримечательность', 'Wifi',
              'Транспорт', 'Заправка', 'Парковка', 'Шоппинг', 'Банкомат', 'Ночная жизнь',
              'Отдых с детьми', 'Банк', 'Развлечения', 'Больница', 'Аптека', 'Полиция',
              'Туалет', 'Почта']
categories_en = ['Where to eat', 'Hotel', 'Groceries', 'Sights', 'Wifi', 'Transport', 'Gas',
                 'Parking', 'Shopping', 'Atm', 'Nightlife', 'Family holiday', 'Bank',
                 'Entertainment', 'Hospital', 'Pharmacy', 'Police', 'Toilet', 'Post']

In [40]:
moscow_x = [37.547149658203125, 37.386474609375, 37.316436767578125, 37.45513916015625,
            37.651519775390625, 37.88360595703125, 37.85614013671875, 37.547149658203125]
moscow_y = [55.93304863776238, 55.88301417578358, 55.76034990679016, 55.598523085310276,
            55.5519418606257, 55.65279803318956, 55.85989956952263, 55.93304863776238]

def in_moscow(x, y):
    """
    Задача о принадлежности точки многоугольнику (МКАД)
    """
    c = 0
    for i in range(len(moscow_x)):
        if (moscow_y[i] <= y < moscow_y[i - 1] or moscow_y[i - 1] <= y < moscow_y[i]) and \
                (x > (moscow_x[i - 1] - moscow_x[i]) *
                 (y - moscow_y[i]) / (moscow_y[i - 1] - moscow_y[i]) + moscow_x[i]):
            c = 1 - c
    if c:
        return True
    return False

In [41]:
with open('raw_search_data.csv', 'r') as csv_file:
    search_reader = csv.DictReader(csv_file, delimiter=';')
    top = Counter()
    top_moscow = Counter()
    top_en = Counter()
    top_en_moscow = Counter()
    top_categories = Counter()
    top_categories_en = Counter()
    user_events = []
    current_uid = None
    moscow = False
    locale = ''
    gps = re.compile(r'\[.*\]')
    for row in search_reader:
        if row['uid'] != current_uid:
            query = find_query(user_events)
            for q in query:
                if q in categories:
                    top_categories[q] += 1
                elif q in categories_en:
                    top_categories_en[q] += 1
                else:
                    if locale.startswith('ru'):
                        top[q] += 1
                        if moscow:
                            top_moscow[q] += 1
                    elif locale.startswith('en'):
                        top_en[q] += 1
                        if moscow:
                            top_en_moscow[q] += 1
            user_events.clear()
            current_uid = row['uid']
            locale = row['locale'] if isinstance(row['locale'], str) else ''
            if isinstance(row['gps'], str) and gps.match(row['gps']):
                x, y = list(map(float, row['gps'].lstrip('[').rstrip(']').split(', ')))
                y = math.degrees(2 * math.atan(math.tanh(0.5 * math.radians(y))))
                moscow = in_moscow(x, y)
        if row['query'] not in user_events:
            user_events.append(row['query'])

In [42]:
top = dict(top.most_common(30))
fig = go.Figure(
    [go.Bar(x=list(top.keys()), y=list(top.values()), name='top')],
    layout_title_text='Топ запросов для устройств с русской локализацией'
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [43]:
top_en = dict(top_en.most_common(30))
fig = go.Figure(
    [go.Bar(x=list(top_en.keys()), y=list(top_en.values()))],
    layout_title_text='Топ запросов для устройств с английской локализацией'
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [44]:
top_moscow = dict(top_moscow.most_common(30))
fig = go.Figure(
    [go.Bar(x=list(top_moscow.keys()), y=list(top_moscow.values()))],
    layout_title_text='Топ запросов для устройств с русской локализацией в Москве'
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [45]:
top_en_moscow = dict(top_en_moscow.most_common(30))
fig = go.Figure(
    [go.Bar(x=list(top_en_moscow.keys()), y=list(top_en_moscow.values()))],
    layout_title_text='Топ запросов для устройств с английской локализацией в Москве'
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [46]:
top_categories = dict(top_categories.most_common(30))
fig = go.Figure(
    [go.Bar(x=list(top_categories.keys()), y=list(top_categories.values()))],
    layout_title_text='Топ запросов по категориям на русском'
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [47]:
top_categories_en = dict(top_categories_en.most_common(30))
fig = go.Figure(
    [go.Bar(x=list(top_categories_en.keys()), y=list(top_categories_en.values()))],
    layout_title_text='Топ запросов по категориям на английском'
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [48]:
from IPython.display import IFrame

# Карта пользователей maps.me
IFrame(src='kepler.gl.html', width=900, height=600)